In [1]:
import pandas as pd
import numpy as np
import sklearn as sk
import matplotlib
import matplotlib.pyplot as plt
import scipy
from subprocess import call
from IPython.display import Image
from sklearn.model_selection import train_test_split
from scipy import stats

%matplotlib inline
matplotlib.style.use('bmh')

data_all_thomas = pd.read_excel('../annotations/data_all_thomas.xlsx')
data_labels_video_features = pd.read_csv('../annotations/combined_labels_video_features.csv')
labels = data_labels_video_features[['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val','gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val']]
labels_max = labels[['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val']]
labels_min = labels[['gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val']]
video_features = data_labels_video_features.drop(['gold_gt_max_aro','gold_gt_max_like','gold_gt_max_val','gold_gt_min_aro','gold_gt_min_like','gold_gt_min_val'],axis=1)

In [2]:
# Applying z normalization
video_features = video_features.apply(stats.zscore)

In [3]:
# Applying L2 normalization
video_features = pd.DataFrame(sk.preprocessing.normalize(video_features, norm='l2',axis=1))

In [4]:
# Splitting and converting train and test data
X_train, X_test, y_train, y_test = train_test_split(video_features, labels['gold_gt_min_aro'], train_size=660,test_size=300,shuffle=False)


X_train = X_train.to_numpy()
X_test = X_test.to_numpy()
y_train = y_train.to_numpy()
y_test = y_test.to_numpy()

In [7]:
# Single ELM
import kernel_elm as elm
from sklearn.metrics import classification_report, confusion_matrix,recall_score


elmk = elm.Extreme_Learning_Machine()
beta = elmk.train(X_train,y_train,weighted=True)
y_predict = elmk.test(X_test)
recall_score(y_test, y_predict,average='macro')

0.4307714430385943

In [9]:
# Multiple ELM
import kernel_elm as elm
from sklearn.metrics import classification_report, confusion_matrix,recall_score


variables = [labels['gold_gt_min_aro'],labels['gold_gt_min_like'],labels['gold_gt_min_val'],data_all_thomas['arousal'],data_all_thomas['likeability'],data_all_thomas['valence']]
hyperparams_c = [100000,10000,1000,100,10,1,0.1,0.01,0.001,0.0001,0.00001]
gammas = [100000,10000,1000,100,10,1,0.1,0.01,0.001,0.0001,0.00001]
report_kelms = pd.DataFrame(columns=['variable_name'] + hyperparams_c)

for variable in variables:
    row = [variable.name]
    for param_c in hyperparams_c:
        X_train, X_test, y_train, y_test = train_test_split(video_features, variable, train_size=660,test_size=300,shuffle=False)
        X_train = X_train.to_numpy()
        X_test = X_test.to_numpy()
        y_train = y_train.to_numpy()
        y_test = y_test.to_numpy()
        kelm = elm.Extreme_Learning_Machine(C=param_c, kernel='linear')
        kelm.train(X_train,y_train, weighted=True)
        y_pred_single = kelm.test(X_test)
        row = row + [recall_score(y_test, y_pred_single,average='macro')]
    report_kelms = report_kelms.append(pd.Series(row,index=report_kelms.columns),ignore_index=True)


report_kelms.to_csv('report_kelm_c_linear_weighted.csv')

In [7]:
report_kelms.to_csv('report_kelm_c_linear.csv')

In [20]:
number_training_rows = X_train.shape[0]

W = np.identity(number_training_rows)
ClassHistogram = np.histogram(y_train,bins=[1,2,3,4])[0]
ClassHistogram = np.divide(1,ClassHistogram)
for i in range(0,number_training_rows):
    W[i,i]=ClassHistogram[y_train[i]-1]

In [12]:
y_train

array([2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 3, 3, 1, 2, 2, 2, 2, 2, 1, 3,
       3, 2, 2, 2, 1, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 2, 3, 1, 3, 1, 2,
       2, 1, 2, 2, 2, 2, 3, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 3, 2, 3, 2, 2, 2, 2, 2, 3, 3, 3, 3, 2, 2, 1, 2, 1, 2, 2,
       3, 1, 3, 3, 3, 2, 3, 3, 2, 2, 2, 3, 3, 2, 2, 2, 2, 2, 2, 3, 1, 1,
       2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 3, 1, 2, 2, 2, 3, 1, 3, 2, 2, 3,
       2, 2, 2, 2, 3, 3, 2, 2, 1, 3, 3, 2, 2, 2, 1, 1, 3, 3, 2, 2, 2, 3,
       2, 2, 2, 3, 1, 2, 2, 2, 2, 2, 3, 3, 2, 3, 2, 2, 2, 2, 3, 2, 2, 2,
       3, 2, 2, 1, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 3, 2, 2,
       1, 3, 2, 3, 1, 2, 2, 3, 3, 2, 2, 1, 1, 2, 3, 3, 3, 2, 2, 2, 2, 1,
       2, 3, 3, 3, 2, 2, 3, 2, 2, 3, 3, 2, 3, 2, 2, 2, 2, 1, 2, 2, 2, 3,
       1, 3, 2, 2, 2, 3, 3, 2, 3, 3, 2, 2, 2, 2, 2, 3, 3, 3, 2, 2, 2, 1,
       1, 1, 3, 2, 3, 2, 2, 2, 3, 3, 3, 2, 2, 3, 2, 2, 2, 1, 2, 1, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 1, 3, 2, 2, 2, 2, 2, 2,